<h1>Implementation 1</h1>
<br><b>Le Gall 5/3 implementation in pywt</b>
<br><br><b>Notes:</b>
<li>Pywt wavedec2 method offers decomposition of data till n levels.
<li>Le Gall 5/3 is referred as 'bior2.2'
<li>Pywt does not support lifting scheme required by Le Gall 5/3 integer lifting scheme mentioned in HanQiu[2021].
<li>This results in data not being perfectly reconstructed and leading to lossy reconstruction.

In [39]:
import pywt
import numpy as np

image_file = 'avu.png'
file_path = f'data/images/{image_file}'
rows = 8
cols = 8  
chunk_size = rows * cols  
wavelet = 'bior2.2'
count = 0
with open(file_path, 'rb') as file:
    while True:
        chunk = file.read(chunk_size)
        if not chunk:
            break  # end of file

        remaining_bytes = chunk_size - (len(chunk) % chunk_size)
        if remaining_bytes < chunk_size:
            padding = bytearray([0] * remaining_bytes)
            chunk += padding
        
        chunk_uint8 = np.frombuffer(chunk, dtype=np.uint8)
        chunk_uint8_2d = chunk_uint8.reshape((-1, cols))
        print(f'Original Data:\n{chunk_uint8_2d}')
        
        
        coeffs = pywt.wavedec2(chunk_uint8_2d, wavelet=wavelet, level=2)

        LL2, (LH2, HL2, HH2), (LH1, HL1, HH1) = coeffs
        
        
        rec_chunk = np.uint8(pywt.waverec2(coeffs=coeffs, wavelet=wavelet))
        print(f'Reconstructed Data:\n{rec_chunk}\n')   
        print(f"Perfect data reconstruction... {np.array_equal(chunk, rec_chunk)}\n")
        break


<h1>Implementation 2</h1>
<br><b>Integer-To-Integer Le Gall 5/3 wavelet transform</b>
<br><br><b>Notes:</b>
<li>This implementation of the lossless transform supports 1 dimensional input only. 
<li>Input data can be seen reconstructed perfectly.
<li>Available at <a href=https://github.com/christofer-f/IntegerWaveletTransform>github link</a>

In [40]:
from math import floor, ceil
import numpy as np
import random

def iwt53(c):    
    s = c[0::2]
    d = c[1::2]
    l = len(s)

    a = d[0:l-1] - np.floor(0.5*(s[0:l-1]+s[1:l])) 
    b = d[l-1] - s[l-1] 
    d = np.concatenate((a, b), axis=None)   

    a = s[0] + np.floor(0.5*d[0] + 0.5)
    b = s[1:l] + np.floor(0.25*(d[1:l] + d[0:l-1]) + 0.5)
    s = np.concatenate((a, b), axis=None)
    
    return s, d


def iiwt53(s,d):
    l = len(s)

    a = s[0] - np.floor(0.5*d[0] + 0.5)
    b = s[1:l] - np.floor(0.25*(d[1:l] + d[0:l-1]) + 0.5)
    s = np.concatenate((a, b), axis=None)

    a = d[0:l-1] + np.floor(0.5*(s[0:l-1]+s[1:l])) 
    b = d[l-1] + s[l-1] 
    d = np.concatenate((a, b), axis=None)   

    c2 = np.column_stack((s,d)).ravel()

    return c2


def main():    
    
    c = np.arange(64)
    print(f'Original Data:\n{c}\n')
    s,d = iwt53(c)
    print(f'approx: {s}\ndetails: {d}\n')
    c2 = iiwt53(s,d)
    print(f'Reconstructed Data: {c2}\n')
    print(f"Perfect data reconstruction... {np.array_equal(c, c2)}\n")

if __name__ == "__main__":
    main()

<h1>Implementation 3</h1>
<br><b>Integer-To-Integer Le Gall 5/3 wavelet transform</b>
<br><br><b>Notes:</b>
<li>This implementation too supports 1 dimensional input only. 
<li>Input data can be seen reconstructed perfectly.
<li>Transformed data (approx, details) match with the prior impelementation
<li>Available at <a href=https://gist.github.com/101arrowz/89e0caa0ab0894e8335ae2d97a3c5562>github link</a>


In [41]:
# Lifting scheme version of the discrete wavelet transform in JPEG 2000

# The approximation and detail coefficients are interleaved in the output of dwt (even 
# indices for approx, odd for detail). In practice you may need to separate them afterwards

# idwt is esentially reversing the order of the lifting scheme steps in dwt and
# flipping the sign of operations(i.e. addition becomes subtraction and vice versa)

# approx_len = approximation coefficients length = ceil(total_series_len / 2)
# detail_len = detail coefficients length = floor(total_series_len / 2)

# If you do out the math, the equivalent 1D convolution filters are:
#
# approx analysis: (-1/8, 2/8, 6/8, 2/8, -1/8)
# detail analysis: (-1/2, 1, -1/2)
#
# approx synthesis: (1/2, 1, 1/2)
# detail synthesis: (-1/8, -2/8, 6/8, -2/8, -1/8)
#
# These don't actually work due to the weird use of rounding though.
# Le Gall seems designed specifically for use in a lifting scheme.

import numpy as np

def slow_dwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len

    for i in range(detail_len):
        seq[(i << 1) + 1] -= (seq[i << 1] + seq[min((i << 1) + 2, (approx_len - 1) << 1)]) >> 1
    
    for i in range(approx_len):
        seq[i << 1] += (seq[max((i << 1) - 1, 1)] + seq[min((i << 1) + 1, ((detail_len - 1) << 1) + 1)] + 2) >> 2

    return seq

def slow_idwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len
    
    for i in range(approx_len):
        seq[i << 1] -= (seq[max((i << 1) - 1, 1)] + seq[min((i << 1) + 1, ((detail_len - 1) << 1) + 1)] + 2) >> 2

    for i in range(detail_len):
        seq[(i << 1) + 1] += (seq[i << 1] + seq[min((i << 1) + 2, (approx_len - 1) << 1)]) >> 1
    
    return seq

def dwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len

    for i in range(1, detail_len):
        seq[(i << 1) - 1] -= (seq[(i - 1) << 1] + seq[i << 1]) >> 1
    seq[(detail_len << 1) - 1] -= (seq[(detail_len - 1) << 1] + seq[(approx_len - 1) << 1]) >> 1

    seq[0] += (seq[1] + seq[1] + 2) >> 2
    for i in range(1, approx_len - 1):
        seq[i << 1] += (seq[(i << 1) - 1] + seq[(i << 1) + 1] + 2) >> 2
    seq[(approx_len - 1) << 1] += (seq[(approx_len << 1) - 3] + seq[(detail_len << 1) - 1] + 2) >> 2

    return seq

def idwt(seq):
    seq = np.copy(seq)
    n = seq.shape[0]

    detail_len = n >> 1
    approx_len = n - detail_len
    
    seq[0] -= (seq[1] + seq[1] + 2) >> 2
    for i in range(1, approx_len - 1):
        seq[i << 1] -= (seq[(i << 1) - 1] + seq[(i << 1) + 1] + 2) >> 2
    seq[(approx_len - 1) << 1] -= (seq[(approx_len << 1) - 3] + seq[(detail_len << 1) - 1] + 2) >> 2

    for i in range(1, detail_len):
        seq[(i << 1) - 1] += (seq[(i - 1) << 1] + seq[i << 1]) >> 1
    seq[(detail_len << 1) - 1] += (seq[(detail_len - 1) << 1] + seq[(approx_len - 1) << 1]) >> 1

    return seq

seq = np.arange(64)

print(f'Original Data:\n{seq}\n')

coeffs = dwt(seq)
approx = coeffs[::2]
detail = coeffs[1::2]
print(f'approx: {approx}\ndetail: {detail}\n')
reconstructed = idwt(coeffs)
print(f'Reconstructed Data:\n{reconstructed}\n')
print(f"\nPerfect data reconstruction... {np.array_equal(seq, reconstructed)}\n")